In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import glob
import os
import openpyxl
from IPython.display import display, HTML
from typing import Dict

# Import project utilities
import sys

sys.path.append("../../../")

# Visualization settings
plt.style.use("seaborn-v0_8-whitegrid")
%matplotlib inline
sns.set_palette("viridis")
plt.rcParams["figure.figsize"] = (12, 8)
pd.set_option("display.max_columns", None)

# For reproducibility
np.random.seed(42)

# Configuration
XLSX_DATA_PATH = "../../../data/raw/aquaculture/aerator_imports/bernauer.xlsx"
OUTPUT_DIR = "../../../data/processed/bernauer/df2/"

# Global visualization settings
used_colors = set()
site_color_mapping = {}
available_css4_colors = list(mcolors.CSS4_COLORS.keys())


def load_all_excel_sheets(data_path: str) -> Dict[str, pd.DataFrame]:
    """
    Load all Excel sheets and return them as separate DataFrames.

    Args:
        data_path (str): Path pattern for Excel files

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of DataFrames keyed by sheet names
    """
    try:
        files = glob.glob(data_path)
        if not files:
            print("No Excel files found matching the pattern.")
            return {}

        all_dataframes = {}

        for file in files:
            print(f"Loading data from {file}")

            # Load workbook with openpyxl for inspection
            wb = openpyxl.load_workbook(file, data_only=True, read_only=True)
            file_basename = os.path.basename(file)

            print(f"\nWorkbook: {file_basename}")
            print(
                f"Contains {len(wb.sheetnames)} sheets: {', '.join(wb.sheetnames)}"
            )

            # Process all sheets
            for i, sheet_name in enumerate(wb.sheetnames):
                df = _load_sheet_with_header_detection(
                    file, sheet_name, wb[sheet_name]
                )

                if df is not None and not df.empty:
                    df_key = f"df{i + 1}"
                    all_dataframes[df_key] = df
                    print(
                        f"  Loaded {df_key} from sheet '{sheet_name}' with shape {df.shape}"
                    )

            wb.close()

        return all_dataframes

    except Exception as e:
        print(f"Error loading data: {e}")
        raise


def _load_sheet_with_header_detection(
    file_path: str, sheet_name: str, sheet
) -> pd.DataFrame:
    """
    Load a sheet with automatic header detection.

    Args:
        file_path (str): Path to the Excel file
        sheet_name (str): Name of the sheet
        sheet: Openpyxl sheet object

    Returns:
        pd.DataFrame: Loaded DataFrame or None if sheet is empty
    """
    try:
        # Check if sheet is empty
        if not sheet.max_row or not sheet.max_column:
            print(
                f"  Warning: Sheet '{sheet_name}' appears to be empty or corrupted"
            )
            return None

        print(f"\n  Sheet: '{sheet_name}'")
        print(
            f"  Dimensions: {sheet.max_row} rows x {sheet.max_column} columns"
        )

        # Check for merged cells
        try:
            merged_cells = list(sheet.merged_cells.ranges)
            if merged_cells:
                print(f"  Contains {len(merged_cells)} merged cell ranges")
        except AttributeError:
            print("  Note: Cannot check merged cells in read-only mode")

        # Load initial DataFrame without headers
        df = pd.read_excel(
            file_path,
            sheet_name=sheet_name,
            header=None,
            na_values=["NA", "N/A", ""],
            keep_default_na=True,
        )

        # Detect header row
        header_row = _detect_header_row(df)

        if header_row is not None:
            # Reload with detected header
            df = pd.read_excel(
                file_path,
                sheet_name=sheet_name,
                header=header_row,
                na_values=["NA", "N/A", ""],
                keep_default_na=True,
            )
            print(f"  Detected header at row {header_row + 1}")

        return df

    except Exception as e:
        print(f"  Error loading sheet '{sheet_name}': {e}")
        return None


def _detect_header_row(df: pd.DataFrame) -> int:
    """
    Detect the most likely header row in a DataFrame.

    Args:
        df (pd.DataFrame): DataFrame to analyze

    Returns:
        int: Index of the header row, or None if not found
    """
    for i in range(min(10, len(df))):
        str_count = sum(1 for x in df.iloc[i] if isinstance(x, str))
        if str_count > 0.5 * df.shape[1]:  # More than half are strings
            return i
    return None


# Step 1: Load all sheets
print("\n" + "=" * 60)
print("STEP 1: LOADING DATA FROM ALL SHEETS")
print("=" * 60)
all_dataframes = load_all_excel_sheets(XLSX_DATA_PATH)

if not all_dataframes:
    print("❌ No data loaded. Please check the file path and try again.")
    sys.exit(1)

# Extract individual DataFrames for easier access
df1 = all_dataframes.get("df1")
df2 = all_dataframes.get("df2")
df3 = all_dataframes.get("df3")
df4 = all_dataframes.get("df4")
df5 = all_dataframes.get("df5")

print(f"\n✅ Successfully loaded {len(all_dataframes)} DataFrames:")
for df_name in all_dataframes.keys():
    print(f"  - {df_name}: {all_dataframes[df_name].shape}")


def clean_single_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Apply cleaning to a single DataFrame.

    Args:
        df (pd.DataFrame): DataFrame to clean

    Returns:
        pd.DataFrame: Cleaned DataFrame
    """
    print("Starting data cleaning process...")

    # Apply cleaning steps in order
    df = standardize_headers_single(df)
    df = remove_empty_rows_and_cols_single(df)
    df = handle_nan_values_single(df)
    df = title_case_columns_single(df)

    print("Data cleaning completed!")
    return df


def standardize_headers_single(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize column headers for a single DataFrame."""
    print("STANDARDIZING DATAFRAME HEADERS")

    # Handle unnamed columns
    df = _fix_unnamed_columns(df)

    # Standardize date columns
    df = _standardize_date_columns(df)

    print("Header standardization complete!")
    return df


def remove_empty_rows_and_cols_single(df: pd.DataFrame) -> pd.DataFrame:
    """Remove completely empty rows and columns from DataFrame."""
    original_shape = df.shape
    df = df.dropna(how="all").dropna(axis=1, how="all")
    print(f"Cleaned DataFrame: {original_shape} -> {df.shape}")
    return df


def handle_nan_values_single(df: pd.DataFrame) -> pd.DataFrame:
    """Analyze and selectively handle NaN values in DataFrame."""
    print(f"\n{'-' * 50}")
    print("NaN analysis for DataFrame:")

    # Calculate NaN statistics
    total_cells = df.shape[0] * df.shape[1]
    nan_count = df.isna().sum().sum()
    nan_percentage = (nan_count / total_cells) if total_cells > 0 else 0

    print(
        f"- Total NaN values: {nan_count} ({nan_percentage:.1%} of all cells)"
    )

    # Handle high-NaN columns
    df = _remove_high_nan_columns(df)

    # Handle critical row removal
    df = _remove_critical_nan_rows(df)

    print(f"- Final shape after analysis: {df.shape}")
    return df


def title_case_columns_single(df: pd.DataFrame) -> pd.DataFrame:
    """Apply title case to column names."""
    df.columns = df.columns.str.title()
    return df


def _fix_unnamed_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Fix unnamed columns by replacing them with meaningful names."""
    columns = df.columns.tolist()
    for i, col in enumerate(columns):
        col_str = str(col)
        if col_str.startswith("Unnamed") or col_str.isdigit():
            columns[i] = f"Column_{i}"
    df.columns = columns
    return df


def _standardize_date_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Standardize date column names."""
    date_column_mapping = {
        "fecha_embarque": "Fecha De Embarque",
        "fecha_llegada": "Fecha De Llegada",
        "fecha_ingreso": "Fecha Ingreso Sistema",
        "fecha_pago": "Fecha Pago",
        "fecha_liquidacion": "Fecha De Liquidación",
        "fecha_aforo": "Fecha Aforo",
        "fecha_salida": "Fecha Salida",
    }

    # Apply mapping if columns exist, handling non-string column names
    new_columns = []
    for col in df.columns:
        # Convert column to string if it's not already
        col_str = str(col)
        # Apply mapping
        mapped_col = date_column_mapping.get(
            col_str.lower().replace(" ", "_"), col
        )
        new_columns.append(mapped_col)

    df.columns = new_columns
    return df


def _remove_high_nan_columns(
    df: pd.DataFrame, threshold: float = 0.8
) -> pd.DataFrame:
    """Remove columns with high percentage of NaN values."""
    nan_percentage = df.isnull().sum() / len(df)
    high_nan_cols = nan_percentage[nan_percentage > threshold].index.tolist()

    if high_nan_cols:
        print(
            f"- Removing {len(high_nan_cols)} columns with >{threshold * 100}% NaN values"
        )
        df = df.drop(columns=high_nan_cols)

    return df


def _remove_critical_nan_rows(df: pd.DataFrame) -> pd.DataFrame:
    """Remove rows where critical columns are all NaN."""
    critical_columns = [
        "Us$ Fob",
        "Cantidad",
        "Descripcion Producto Comercial",
    ]
    existing_critical = [col for col in critical_columns if col in df.columns]

    if existing_critical:
        before_count = len(df)
        df = df.dropna(subset=existing_critical, how="all")
        removed = before_count - len(df)
        if removed > 0:
            print(f"- Removed {removed} rows with all critical columns as NaN")

    return df


def _print_single_dataframe_info(name: str, df: pd.DataFrame) -> None:
    """Print comprehensive information about a DataFrame."""
    print(f"\n{'-' * 50}")
    print(f"DataFrame: {name}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")

    # Memory usage
    memory_mb = df.memory_usage(deep=True).sum() / 1024**2
    print(f"Memory usage: {memory_mb:.2f} MB")

    # Data types summary
    dtype_counts = df.dtypes.value_counts()
    print(f"Data types: {dict(dtype_counts)}")

    # Missing values summary
    missing_summary = df.isnull().sum()
    missing_cols = missing_summary[missing_summary > 0]
    if len(missing_cols) > 0:
        print(f"Columns with missing values: {len(missing_cols)}")
        print(f"Total missing values: {missing_summary.sum()}")
    else:
        print("No missing values found")

    print(f"{'-' * 50}")


def clean_multiple_dataframes(
    dataframes_dict: Dict[str, pd.DataFrame],
) -> Dict[str, pd.DataFrame]:
    """
    Apply cleaning to multiple DataFrames.

    Args:
        dataframes_dict (Dict[str, pd.DataFrame]): Dictionary of DataFrames to clean

    Returns:
        Dict[str, pd.DataFrame]: Dictionary of cleaned DataFrames
    """
    cleaned_dataframes = {}

    for df_name, df in dataframes_dict.items():
        print(f"\nCleaning {df_name}...")
        cleaned_df = clean_single_dataframe(df)
        cleaned_dataframes[df_name] = cleaned_df

    return cleaned_dataframes


# Step 2: Clean all data
print("\n" + "=" * 60)
print("STEP 2: CLEANING ALL DATAFRAMES")
print("=" * 60)
cleaned_dataframes = clean_multiple_dataframes(all_dataframes)

# Update individual DataFrames with cleaned versions
df1 = cleaned_dataframes.get("df1")
df2 = cleaned_dataframes.get("df2")
df3 = cleaned_dataframes.get("df3")
df4 = cleaned_dataframes.get("df4")
df5 = cleaned_dataframes.get("df5")

print(f"\n✅ Successfully cleaned {len(cleaned_dataframes)} DataFrames")



STEP 1: LOADING DATA FROM ALL SHEETS
Loading data from ../../../data/raw/aquaculture/aerator_imports/bernauer.xlsx

Workbook: bernauer.xlsx
Contains 5 sheets: 工作表1, Sheet1, Sheet2, Sheet3, Sheet4

  Sheet: '工作表1'
  Dimensions: 33 rows x 10 columns
  Note: Cannot check merged cells in read-only mode
  Loaded df1 from sheet '工作表1' with shape (33, 8)

  Sheet: 'Sheet1'
  Dimensions: 51 rows x 3 columns
  Note: Cannot check merged cells in read-only mode
  Detected header at row 1
  Loaded df2 from sheet 'Sheet1' with shape (50, 3)

  Sheet: 'Sheet2'
  Dimensions: 30 rows x 11 columns
  Note: Cannot check merged cells in read-only mode
  Loaded df3 from sheet 'Sheet2' with shape (30, 11)

  Sheet: 'Sheet3'
  Dimensions: 4 rows x 7 columns
  Note: Cannot check merged cells in read-only mode
  Detected header at row 1
  Loaded df4 from sheet 'Sheet3' with shape (3, 7)

  Sheet: 'Sheet4'
  Dimensions: 42 rows x 10 columns
  Note: Cannot check merged cells in read-only mode
  Loaded df5 from 

In [2]:
def detect_data_structure(df: pd.DataFrame) -> Dict[str, any]:
    """
    Analyze DataFrame structure to identify header rows, data sections, and metadata.

    Args:
        df (pd.DataFrame): Raw DataFrame to analyze

    Returns:
        Dict containing structure information
    """
    structure_info = {
        "header_row": None,
        "data_start_row": None,
        "metadata_rows": [],
        "empty_rows": [],
        "likely_headers": [],
        "data_type": "unknown",
    }

    # Find empty rows
    for idx, row in df.iterrows():
        if row.isna().all():
            structure_info["empty_rows"].append(idx)

    # Look for potential header patterns
    for idx, row in df.iterrows():
        non_null_values = row.dropna().values
        if len(non_null_values) > 0:
            # Check if row contains typical header keywords
            header_keywords = [
                "model",
                "description",
                "price",
                "amount",
                "qty",
                "quantity",
                "unit",
                "mark",
                "id",
                "name",
                "address",
                "tel",
                "email",
            ]

            row_text = " ".join(
                [str(val).lower() for val in non_null_values if pd.notna(val)]
            )

            if any(keyword in row_text for keyword in header_keywords):
                structure_info["likely_headers"].append(idx)

    # Determine data type based on content
    all_text = " ".join(
        [str(val).lower() for val in df.values.flatten() if pd.notna(val)]
    )

    if any(
        term in all_text for term in ["model", "price", "unit price", "amount"]
    ):
        structure_info["data_type"] = "product_catalog"
    elif any(
        term in all_text for term in ["address", "tel", "email", "company"]
    ):
        structure_info["data_type"] = "contact_info"
    elif any(term in all_text for term in ["invoice", "pi. no", "sold to"]):
        structure_info["data_type"] = "invoice_header"
    elif any(
        term in all_text
        for term in ["cubit meter", "qty", "float", "impeller"]
    ):
        structure_info["data_type"] = "shipping_manifest"

    return structure_info


def clean_product_catalog(df: pd.DataFrame) -> pd.DataFrame:
    """Clean product catalog data (like df2)."""
    # Find the actual header row
    header_row_idx = None
    for idx, row in df.iterrows():
        if any(
            "model" in str(val).lower() for val in row.values if pd.notna(val)
        ):
            header_row_idx = idx
            break

    if header_row_idx is not None:
        # Set proper headers
        new_headers = df.iloc[header_row_idx].values
        df_clean = df.iloc[header_row_idx + 1 :].copy()
        df_clean.columns = [
            str(col).strip() if pd.notna(col) else f"Column_{i}"
            for i, col in enumerate(new_headers)
        ]
    else:
        df_clean = df.copy()

    # Clean column names
    df_clean.columns = [
        col.replace(" ", "_").replace("(", "").replace(")", "")
        for col in df_clean.columns
    ]

    # Remove rows that are all NaN
    df_clean = df_clean.dropna(how="all")

    # Convert price columns to numeric
    for col in df_clean.columns:
        if (
            "price" in col.lower()
            or "amount" in col.lower()
            or col.startswith("Column_")
        ):
            df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

    # Reset index
    df_clean = df_clean.reset_index(drop=True)

    return df_clean


def clean_contact_info(df: pd.DataFrame) -> pd.DataFrame:
    """Clean contact information data (like df4)."""
    # Remove rows that are mostly empty
    df_clean = df.dropna(
        thresh=3
    ).copy()  # Keep rows with at least 3 non-null values

    # Try to create meaningful column names based on content
    if len(df_clean) > 0:
        potential_columns = [
            "Customer_ID",
            "Company_Name",
            "Contact_Person",
            "Address",
            "Location",
            "Phone",
            "City",
        ]

        # Assign column names based on available columns
        new_columns = []
        for i, col in enumerate(df_clean.columns):
            if i < len(potential_columns):
                new_columns.append(potential_columns[i])
            else:
                new_columns.append(f"Additional_Info_{i}")

        df_clean.columns = new_columns[: len(df_clean.columns)]

    # Reset index
    df_clean = df_clean.reset_index(drop=True)

    return df_clean


def clean_invoice_header(df: pd.DataFrame) -> pd.DataFrame:
    """Clean invoice header data (like df1)."""
    # This type of data is often metadata - extract key-value pairs
    invoice_data = {}

    for idx, row in df.iterrows():
        row_values = [
            str(val)
            for val in row.values
            if pd.notna(val) and str(val).strip()
        ]

        if len(row_values) >= 2:
            # Look for key-value patterns
            for i in range(0, len(row_values) - 1, 2):
                key = str(row_values[i]).strip()
                value = (
                    str(row_values[i + 1]).strip()
                    if i + 1 < len(row_values)
                    else ""
                )

                if key and value and key not in ["nan", "NaN"]:
                    invoice_data[key] = value

    # Convert to DataFrame
    if invoice_data:
        df_clean = pd.DataFrame([invoice_data])
    else:
        # Fallback: just clean the original structure
        df_clean = df.dropna(how="all").reset_index(drop=True)
        # Remove columns that are mostly empty
        for col in df_clean.columns:
            if df_clean[col].isna().sum() / len(df_clean) > 0.8:
                df_clean = df_clean.drop(columns=[col])

    return df_clean


def clean_shipping_manifest(df: pd.DataFrame) -> pd.DataFrame:
    """Clean shipping manifest data (like df5)."""
    # Find header row
    header_row_idx = None
    for idx, row in df.iterrows():
        if any(
            "qty" in str(val).lower() for val in row.values if pd.notna(val)
        ):
            header_row_idx = idx
            break

    if header_row_idx is not None:
        # Set proper headers
        headers = df.iloc[header_row_idx].values
        df_clean = df.iloc[header_row_idx + 1 :].copy()

        # Create meaningful column names
        new_columns = []
        for i, header in enumerate(headers):
            if pd.notna(header) and str(header).strip():
                new_columns.append(str(header).strip().replace(" ", "_"))
            else:
                # Infer from data pattern
                if i == 0:
                    new_columns.append("Product_Name")
                elif "cubit" in str(headers).lower():
                    new_columns.append(f"Volume_m3_{i}")
                elif any(
                    "qty" in str(h).lower() for h in headers if pd.notna(h)
                ):
                    new_columns.append("Quantity")
                else:
                    new_columns.append(f"Column_{i}")

        df_clean.columns = new_columns
    else:
        df_clean = df.copy()
        # Create default meaningful names
        df_clean.columns = [
            "Product_Name",
            "Unit_Volume_m3",
            "Quantity",
            "Total_Volume_m3",
            "Notes",
        ][: len(df_clean.columns)]

    # Clean data types
    for col in df_clean.columns:
        if (
            "volume" in col.lower()
            or "qty" in col.lower()
            or "quantity" in col.lower()
        ):
            df_clean[col] = pd.to_numeric(df_clean[col], errors="coerce")

    # Remove empty rows and rows with no product name
    df_clean = df_clean.dropna(subset=[df_clean.columns[0]], how="all")
    df_clean = df_clean.reset_index(drop=True)

    return df_clean


def basic_dataframe_clean(df: pd.DataFrame) -> pd.DataFrame:
    """Basic cleaning for unrecognized data types."""
    # Remove completely empty rows and columns
    df_clean = df.dropna(how="all").copy()
    df_clean = df_clean.dropna(axis=1, how="all")

    # Remove columns that are >90% empty
    threshold = len(df_clean) * 0.1  # Keep if at least 10% has data
    df_clean = df_clean.dropna(axis=1, thresh=threshold)

    # Clean column names
    df_clean.columns = [
        f"Column_{i}"
        if str(col).strip() == "" or pd.isna(col)
        else str(col).strip()
        for i, col in enumerate(df_clean.columns)
    ]

    return df_clean.reset_index(drop=True)


# Apply enhanced cleaning to all available DataFrames
print("\n" + "=" * 60)
print("STEP 3: APPLYING ENHANCED CLEANING")
print("=" * 60)

enhanced_dataframes = {}

if df1 is not None:
    print("🔧 Cleaning df1 (Invoice Header)...")
    enhanced_df1 = clean_invoice_header(df1)
    enhanced_dataframes["enhanced_df1"] = enhanced_df1
    print(f"   df1: {df1.shape} -> {enhanced_df1.shape}")

if df2 is not None:
    print("🔧 Cleaning df2 (Product Catalog)...")
    enhanced_df2 = clean_product_catalog(df2)
    enhanced_dataframes["enhanced_df2"] = enhanced_df2
    print(f"   df2: {df2.shape} -> {enhanced_df2.shape}")

if df3 is not None:
    print("🔧 Cleaning df3...")
    enhanced_df3 = basic_dataframe_clean(df3)
    enhanced_dataframes["enhanced_df3"] = enhanced_df3
    print(f"   df3: {df3.shape} -> {enhanced_df3.shape}")

if df4 is not None:
    print("🔧 Cleaning df4 (Contact Info)...")
    enhanced_df4 = clean_contact_info(df4)
    enhanced_dataframes["enhanced_df4"] = enhanced_df4
    print(f"   df4: {df4.shape} -> {enhanced_df4.shape}")

if df5 is not None:
    print("🔧 Cleaning df5 (Shipping Manifest)...")
    enhanced_df5 = clean_shipping_manifest(df5)
    enhanced_dataframes["enhanced_df5"] = enhanced_df5
    print(f"   df5: {df5.shape} -> {enhanced_df5.shape}")

print(
    f"\n✅ Enhanced cleaning completed for {len(enhanced_dataframes)} DataFrames"
)

# Preview each enhanced dataframe
print("\n" + "=" * 60)
print("📊 ENHANCED DATAFRAMES PREVIEW")
print("=" * 60)

for df_name, df in enhanced_dataframes.items():
    print(f"\n📋 {df_name.upper()}:")
    print(f"Shape: {df.shape}")
    if not df.empty:
        print("Columns:", list(df.columns))
        print("Preview:")
        display(HTML(df.head(3).to_html(index=False)))
    print("-" * 40)



STEP 3: APPLYING ENHANCED CLEANING
🔧 Cleaning df1 (Invoice Header)...
   df1: (26, 4) -> (1, 10)
🔧 Cleaning df2 (Product Catalog)...
   df2: (50, 3) -> (50, 3)
🔧 Cleaning df3...
   df3: (29, 3) -> (28, 3)
🔧 Cleaning df4 (Contact Info)...
   df4: (3, 7) -> (3, 7)
🔧 Cleaning df5 (Shipping Manifest)...
   df5: (20, 5) -> (15, 5)

✅ Enhanced cleaning completed for 5 DataFrames

📊 ENHANCED DATAFRAMES PREVIEW

📋 ENHANCED_DF1:
Shape: (1, 10)
Columns: ['SOLD TO:', 'PI. NO：', 'ACUICULTURA BER AQUA AQUIBER S.A.', 'Dirección: JUNIN Y MARCEL LANIADO Nº 439', 'MACHALA ECUADOR', 'MARK', 'UNIT PRICE', 'BR-3008LOP', 'TOTAL CIF', 'Payment:']
Preview:


SOLD TO:,PI. NO：,ACUICULTURA BER AQUA AQUIBER S.A.,Dirección: JUNIN Y MARCEL LANIADO Nº 439,MACHALA ECUADOR,MARK,UNIT PRICE,BR-3008LOP,TOTAL CIF,Payment:
BRAC01,BR-AC202502801,PI. DATE：,P.O.E.：,P.O.D.：,MODEL,AMOUNT(US$),514,5140,30% downpay 70 %T/T in advance against B/L


----------------------------------------

📋 ENHANCED_DF2:
Shape: (50, 3)
Columns: ['Model_', 'Descirption', 'Column_2']
Preview:


Model_,Descirption,Column_2
YH-SSGP,Small Spiral Bevel Gear for Plastic Reducer,3.50
YH-LSGP,Large Spiral Bevel Gear,6.75
YH-SHSP,Small Helical Shaft,3.50


----------------------------------------

📋 ENHANCED_DF3:
Shape: (28, 3)
Columns: ['Column_4', 'Column_9', 'Column_10']
Preview:


...

----------------------------------------

📋 ENHANCED_DF4:
Shape: (3, 7)
Columns: ['Customer_ID', 'Company_Name', 'Contact_Person', 'Address', 'Location', 'Phone', 'City']
Preview:


Customer_ID,Company_Name,Contact_Person,Address,Location,Phone,City
ECEC01,"IMPORTARA ECUASINO ., SA",ATTN: Mr Yu,Centro Empresarial Colon-Empresarial 3- of. 210,ECUADOR,Phone : 593-9484985,GUAYAQUIL
BRBA01,Bernadette Aquacultura LTD,ATTN: Mr Laurent Percy Fleury,Caixa Postal 61 indaial Santa Catarina,Brazil,NaN,NaN
BRAC01,ACUICULTURA BER AQUA AQUIBER S.A.,NaN,Dirección: JUNIN Y MARCEL LANIADO Nº 439,MACHALA ECUADOR,59072935953,GUAYAQUIL


----------------------------------------

📋 ENHANCED_DF5:
Shape: (15, 5)
Columns: ['Teffer_Float', '0.0945', '250', '23.625', 'Column_4']
Preview:


Teffer_Float,0.0945,250,23.625,Column_4
YiYuan Floats,0.0945,258,24.381,NaN
Teffer impeller,0.032,600,19.2,NaN
YiYuan Plastic Reducer with Mechanical Seal,0.02025,300,6.075,x


----------------------------------------


In [3]:
# Analyze df2 by standardizing column names and grouping by code
print("🔧 ANALYZING DF2: PRODUCT CATALOG")
print("=" * 50)

# Create a copy and standardize column names
df2_analysis = enhanced_df2.copy()
df2_analysis.columns = ["Code", "Description", "Cost"]

print(f"Original shape: {df2_analysis.shape}")
print(f"Columns after standardization: {list(df2_analysis.columns)}")

# Convert Cost column to numeric, handling any non-numeric values
df2_analysis["Cost"] = pd.to_numeric(df2_analysis["Cost"], errors="coerce")

# Remove rows where Cost is NaN for grouping analysis
df2_with_cost = df2_analysis.dropna(subset=["Cost"])

print(
    f"\nRows with valid cost data: {len(df2_with_cost)} out of {len(df2_analysis)}"
)

# Extract manufacturer from code (first 2 letters)
df2_with_cost["Manufacturer"] = df2_with_cost["Code"].str[:2]

# Define manufacturer mapping
manufacturer_names = {
    "BR": "BERAQUA",
    "EC": "ECUASINO",
    "TF": "TEFFER",
    "YH": "YIYUAN",
}

# Group by Code and sum costs
code_totals = (
    df2_with_cost.groupby("Code")["Cost"].agg(["sum", "count"]).reset_index()
)
code_totals.columns = ["Code", "Total_Cost", "Count"]

# Sort by Total_Cost in descending order
code_totals_sorted = code_totals.sort_values("Total_Cost", ascending=False)

# Group by manufacturer
manufacturer_totals = (
    df2_with_cost.groupby("Manufacturer")["Cost"]
    .agg(["sum", "count"])
    .reset_index()
)
manufacturer_totals.columns = ["Manufacturer", "Total_Cost", "Component_Count"]
manufacturer_totals["Manufacturer_Name"] = manufacturer_totals[
    "Manufacturer"
].map(manufacturer_names)
manufacturer_totals_sorted = manufacturer_totals.sort_values(
    "Total_Cost", ascending=False
)

print("\n📊 CODE COST ANALYSIS (Top 10):")
print("-" * 40)
display(HTML(code_totals_sorted.head(10).to_html(index=False)))

print("\n📊 MANUFACTURER COST ANALYSIS:")
print("-" * 40)
display(HTML(manufacturer_totals_sorted.to_html(index=False)))

print("\n📈 SUMMARY STATISTICS:")
print(f"Total manufacturers: {len(manufacturer_totals)}")
print(f"Total unique codes: {len(code_totals)}")
print(f"Total cost across all codes: ${code_totals['Total_Cost'].sum():.2f}")
print(
    f"Highest cost code: {code_totals_sorted.iloc[0]['Code']} (${code_totals_sorted.iloc[0]['Total_Cost']:.2f})"
)
print(
    f"Highest cost manufacturer: {manufacturer_totals_sorted.iloc[0]['Manufacturer_Name']} (${manufacturer_totals_sorted.iloc[0]['Total_Cost']:.2f})"
)


🔧 ANALYZING DF2: PRODUCT CATALOG
Original shape: (50, 3)
Columns after standardization: ['Code', 'Description', 'Cost']

Rows with valid cost data: 18 out of 50

📊 CODE COST ANALYSIS (Top 10):
----------------------------------------


/tmp/ipykernel_109581/4144877237.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_with_cost["Manufacturer"] = df2_with_cost["Code"].str[:2]


Code,Total_Cost,Count
BR-3008LOP,514.0,1
YH-2004LMP,300.0,1
YH-FEEDERL,270.0,1
TF-2004LMP,268.0,1
EC-MR5.5MN,267.0,1
YH-2HPMO,83.0,1
YH-1HPMOS,80.0,1
YH-PRD,70.0,1
TF-PRD,60.0,1
YH-FMOL,35.0,1



📊 MANUFACTURER COST ANALYSIS:
----------------------------------------


Manufacturer,Total_Cost,Component_Count,Manufacturer_Name
YH,883.3,13,YIYUAN
BR,514.0,1,BERAQUA
TF,334.0,3,TEFFER
EC,267.0,1,ECUASINO



📈 SUMMARY STATISTICS:
Total manufacturers: 4
Total unique codes: 18
Total cost across all codes: $1998.30
Highest cost code: BR-3008LOP ($514.00)
Highest cost manufacturer: YIYUAN ($883.30)


In [4]:
# Export the model totals to the output directory create directory if it doesn't exist
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")

# Comprehensive Excel export
excel_output_file = os.path.join(OUTPUT_DIR, "bernauer_cost_analysis.xlsx")

# Extract manufacturer from model code (first 2 letters)
df2_with_cost["Manufacturer"] = df2_with_cost["Code"].str[:2]

# Get mapping from manufacturer code to full name
manufacturer_names = {
    "BR": "BERAQUA",
    "EC": "ECUASINO",
    "TF": "TEFFER",
    "YH": "YIYUAN",
}

# Group by manufacturer
manufacturer_totals = (
    df2_with_cost.groupby("Manufacturer")["Cost"]
    .agg(["sum", "count"])
    .reset_index()
)
manufacturer_totals.columns = ["Manufacturer", "Total_Cost", "Component_Count"]

# Add full manufacturer name
manufacturer_totals["Manufacturer_Name"] = manufacturer_totals[
    "Manufacturer"
].map(manufacturer_names)

# Sort by Total_Cost in descending order
manufacturer_totals_sorted = manufacturer_totals.sort_values(
    "Total_Cost", ascending=False
)


# Function to prepare detailed manufacturer breakdown from actual component data
def prepare_manufacturer_breakdown(manufacturer_code, df_components):
    """
    Prepare a detailed breakdown for a manufacturer using actual component data.

    Args:
        manufacturer_code (str): The code of the manufacturer (BR, EC, TF, YH)
        df_components (DataFrame): DataFrame containing manufacturer component data

    Returns:
        DataFrame: Enhanced manufacturer breakdown with components and summary
    """
    # Sort by Cost in descending order
    df_sorted = df_components.sort_values("Cost", ascending=False).copy()

    # Calculate total cost
    total_cost = df_sorted["Cost"].sum()

    # Add percentage column showing contribution to total manufacturer cost
    df_sorted["Percentage"] = (df_sorted["Cost"] / total_cost * 100).round(2)
    df_sorted["Percentage"] = df_sorted["Percentage"].astype(str) + "%"

    # Add summary row
    manufacturer_name = manufacturer_names.get(
        manufacturer_code, manufacturer_code
    )
    summary_row = pd.DataFrame(
        {
            "Code": ["TOTAL"],
            "Description": [f"Total for {manufacturer_name}"],
            "Cost": [total_cost],
            "Percentage": ["100.00%"],
            "Manufacturer": [manufacturer_code],
        }
    )

    # Combine components and summary
    df_combined = pd.concat([df_sorted, summary_row], ignore_index=True)

    return df_combined


with pd.ExcelWriter(excel_output_file, engine="openpyxl") as writer:
    # Sheet 1: Summary by Manufacturer
    print("📊 Creating Manufacturer Summary sheet...")
    manufacturer_totals_sorted.to_excel(
        writer, sheet_name="Manufacturer_Summary", index=False
    )

    # Sheet 2: All Models Summary
    print("📊 Creating Model Summary sheet...")
    # Group by individual model code
    model_totals = (
        df2_with_cost.groupby("Code")["Cost"]
        .agg(["sum", "count"])
        .reset_index()
    )
    model_totals.columns = ["Code", "Total_Cost", "Component_Count"]
    model_totals_sorted = model_totals.sort_values(
        "Total_Cost", ascending=False
    )
    model_totals_sorted.to_excel(
        writer, sheet_name="Model_Summary", index=False
    )

    # Sheet 3: Detailed Insights
    print("📊 Creating Insights sheet...")
    insights_data = {
        "Metric": [
            "Total Manufacturers",
            "Total Model Codes",
            "Total Components Analyzed",
            "Components with Cost Data",
            "Total Cost (USD)",
            "Highest Cost Manufacturer",
            "Highest Cost Manufacturer Amount (USD)",
            "Lowest Cost Manufacturer",
            "Lowest Cost Manufacturer Amount (USD)",
            "Highest Cost Component",
            "Highest Cost Component Amount (USD)",
        ],
        "Value": [
            len(manufacturer_totals),
            len(model_totals),
            len(df2_analysis),
            len(df2_with_cost),
            f"${df2_with_cost['Cost'].sum():.2f}",
            manufacturer_totals_sorted.iloc[0]["Manufacturer_Name"],
            f"${manufacturer_totals_sorted.iloc[0]['Total_Cost']:.2f}",
            manufacturer_totals_sorted.iloc[-1]["Manufacturer_Name"],
            f"${manufacturer_totals_sorted.iloc[-1]['Total_Cost']:.2f}",
            df2_with_cost.sort_values("Cost", ascending=False).iloc[0]["Code"],
            f"${df2_with_cost.sort_values('Cost', ascending=False).iloc[0]['Cost']:.2f}",
        ],
    }
    insights_df = pd.DataFrame(insights_data)
    insights_df.to_excel(writer, sheet_name="Insights", index=False)

    # Sheet 4: All Components List
    print("📊 Creating All Components List sheet...")
    # Sort by Manufacturer then by Cost
    all_components_df = df2_with_cost.sort_values(
        ["Manufacturer", "Cost"], ascending=[True, False]
    )
    all_components_df.to_excel(
        writer, sheet_name="All_Components", index=False
    )

    # Individual sheets for each manufacturer with detailed breakdowns
    print("📊 Creating manufacturer breakdown sheets...")

    # Process each manufacturer
    for i, manufacturer_row in enumerate(
        manufacturer_totals_sorted.to_dict("records"), 1
    ):
        manufacturer_code = manufacturer_row["Manufacturer"]
        manufacturer_name = manufacturer_row["Manufacturer_Name"]

        # Get components for this manufacturer from the dataset
        manufacturer_components = df2_with_cost[
            df2_with_cost["Manufacturer"] == manufacturer_code
        ].copy()

        # Prepare detailed breakdown
        manufacturer_detail_df = prepare_manufacturer_breakdown(
            manufacturer_code, manufacturer_components
        )

        # Clean sheet name for Excel
        sheet_name = f"{manufacturer_name}"

        # Write to Excel
        manufacturer_detail_df.to_excel(
            writer, sheet_name=sheet_name, index=False
        )

        print(
            f"   ✓ {manufacturer_code} ({manufacturer_name}): {len(manufacturer_components)} components, Total: ${manufacturer_row['Total_Cost']:.2f}"
        )

print(f"\n✅ Comprehensive analysis exported to: {excel_output_file}")
print("📋 Excel file contains:")
print("   • Manufacturer_Summary sheet: Overall manufacturer cost summary")
print("   • Model_Summary sheet: Summary by individual model code")
print("   • Insights sheet: Key metrics and statistics")
print(
    "   • All_Components: Complete list of all components sorted by manufacturer"
)
print(
    f"   • {len(manufacturer_totals)} manufacturer sheets: Detailed component breakdowns for each manufacturer"
)

# Also export CSV summaries for quick reference
manufacturer_csv = os.path.join(OUTPUT_DIR, "manufacturer_totals.csv")
manufacturer_totals_sorted.to_csv(manufacturer_csv, index=False)
model_csv = os.path.join(OUTPUT_DIR, "model_totals.csv")
model_totals_sorted.to_csv(model_csv, index=False)
print(f"   • Manufacturer summary: {manufacturer_csv}")
print(f"   • Model summary: {model_csv}")


📊 Creating Manufacturer Summary sheet...
📊 Creating Model Summary sheet...
📊 Creating Insights sheet...
📊 Creating All Components List sheet...
📊 Creating manufacturer breakdown sheets...
   ✓ YH (YIYUAN): 13 components, Total: $883.30
   ✓ BR (BERAQUA): 1 components, Total: $514.00
   ✓ TF (TEFFER): 3 components, Total: $334.00
   ✓ EC (ECUASINO): 1 components, Total: $267.00

✅ Comprehensive analysis exported to: ../../../data/processed/bernauer/df2/bernauer_cost_analysis.xlsx
📋 Excel file contains:
   • Manufacturer_Summary sheet: Overall manufacturer cost summary
   • Model_Summary sheet: Summary by individual model code
   • Insights sheet: Key metrics and statistics
   • All_Components: Complete list of all components sorted by manufacturer
   • 4 manufacturer sheets: Detailed component breakdowns for each manufacturer
   • Manufacturer summary: ../../../data/processed/bernauer/df2/manufacturer_totals.csv
   • Model summary: ../../../data/processed/bernauer/df2/model_totals.csv


/tmp/ipykernel_109581/2568440404.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2_with_cost["Manufacturer"] = df2_with_cost["Code"].str[:2]
